In [1]:
import gym
%pylab inline
import pylab
import numpy as np

Populating the interactive namespace from numpy and matplotlib


# Maximum Entropy Inverse Reinforcement Learning (MaxEntIRL)

The Mountain Car problem is "on a one-dimensional track, positioned between two “mountains”. The goal is to drive up the mountain on the right; however, the car’s engine is not strong enough to scale the mountain in a single pass. Therefore, the only way to succeed is to drive back and forth to build up momentum."

<img src='https://miro.medium.com/max/1104/1*JjBfoFrKCoBxlraVZaEshw.jpeg'>

The car’s state, at any point in time, is given by a vector containing its horizonal position and velocity. The car commences each episode stationary, at the bottom of the valley between the hills (at position, x, approximately -0.5), and the episode ends when either the car reaches the flag (position > 0.5) or after 200 moves.

Your state, s, is a 2-dim vector that will be (position, velocity) or (x,v) of (-0.5,0) in the beginning because the velocity is 0 and x = -0.5 is the valley between the two mountains which is left of center (there is about a distance of 1.1 in front of you and a distance of -0.7 behind you). Only the x-axis is part of the state, the elevation (y-axis) and gravity is implicit. 

The velocity is positive is you are moving forward to the right, and negative if you are moving to the left. 

action 0 is do nothing, action 1 is run the car in reverse towards the left mountain located at `state[0]=-1.2`,
action 2 is run the car forward towards `state[0]=0.6`


In [2]:
# # MACROS
Push_Left = 0
No_Push = 1
Push_Right = 2

In [3]:
# Initialize environment and reset it to start at valley with 0 velocity
env = gym.make('MountainCar-v0')
env.reset() # array([-0.44943939,  0.        ])

# experiment with constantly moving in one direction or the other to see what happens
const_action = No_Push
for t in range(100):
    update = env.step(const_action)
    if t % 5 == 0:
        print(update)
    
# if you do nothing you jus swing side to side, but your position stays in (-0.64,-0.4)
# your velocity stays in (-0.01,+0.01)

(array([-5.52728967e-01,  2.19840917e-04]), -1.0, False, {})
(array([-0.54844612,  0.00126223]), -1.0, False, {})
(array([-0.53957812,  0.0020721 ]), -1.0, False, {})
(array([-0.52776001,  0.00249972]), -1.0, False, {})
(array([-0.51517341,  0.00246575]), -1.0, False, {})
(array([-0.50414268,  0.00197649]), -1.0, False, {})
(array([-0.49670336,  0.0011227 ]), -1.0, False, {})
(array([-4.94226444e-01,  6.21254728e-05]), -1.0, False, {})
(array([-0.49716795, -0.00100988]), -1.0, False, {})
(array([-0.50498629, -0.00189591]), -1.0, False, {})
(array([-0.51624046, -0.00243232]), -1.0, False, {})
(array([-0.52885349, -0.00251966]), -1.0, False, {})
(array([-0.5404961, -0.0021417]), -1.0, False, {})
(array([-0.54901924, -0.00136859]), -1.0, False, {})
(array([-5.52851654e-01, -3.43329022e-04]), -1.0, False, {})
(array([-0.55128763,  0.00074513]), -1.0, False, {})
(array([-0.5446151 ,  0.00169639]), -1.0, False, {})
(array([-0.53406347,  0.00233495]), -1.0, False, {})
(array([-0.52157949,  0.

In [4]:
# Initialize environment and reset it to start at valley with 0 velocity
env = gym.make('MountainCar-v0')
env.reset() # array([-0.44943939,  0.        ])

# experiment with constantly moving in one direction or the other to see what happens
# constantly do nothing
const_action = No_Push
for t in range(100):
    update = env.step(const_action)
    if t % 5 == 0:
        print(update)
    
# if you do nothing you jus swing side to side, but your position stays in (-0.64,-0.4)
# your velocity stays in (-0.01,+0.01)

(array([-0.43772489, -0.00064161]), -1.0, False, {})
(array([-0.45023362, -0.00368832]), -1.0, False, {})
(array([-0.476185 , -0.0060695]), -1.0, False, {})
(array([-0.51085975, -0.00734188]), -1.0, False, {})
(array([-0.54788467, -0.00726095]), -1.0, False, {})
(array([-0.58043168, -0.00584245]), -1.0, False, {})
(array([-0.6025358 , -0.00335815]), -1.0, False, {})
(array([-6.10192431e-01, -2.68653394e-04]), -1.0, False, {})
(array([-0.60202659,  0.00286897]), -1.0, False, {})
(array([-0.57950494,  0.0054904 ]), -1.0, False, {})
(array([-0.54670929,  0.00711236]), -1.0, False, {})
(array([-0.50965213,  0.00742559]), -1.0, False, {})
(array([-0.47516783,  0.00636919]), -1.0, False, {})
(array([-0.4495929 ,  0.00414696]), -1.0, False, {})
(array([-0.43757634,  0.001175  ]), -1.0, False, {})
(array([-0.44127993, -0.0020076 ]), -1.0, False, {})
(array([-0.46003917, -0.00482979]), -1.0, False, {})
(array([-0.49046494, -0.00677595]), -1.0, False, {})
(array([-0.52699596, -0.00747833]), -1.0

In [5]:
# Initialize environment and reset it to start at valley with 0 velocity
env = gym.make('MountainCar-v0')
env.reset() # array([-0.44943939,  0.        ])

# experiment with constantly moving in one direction or the other to see what happens
# constantly go right
const_action = Push_Right
for t in range(100):
    update = env.step(const_action)
    if t % 5 == 0:
        print(update)
    
# if you constantly go right,  your position stays in (-0.52,-0.24)
# your velocity stays in (-0.011,+0.011)

(array([-0.58871941,  0.00149631]), -1.0, False, {})
(array([-0.55955899,  0.00859576]), -1.0, False, {})
(array([-0.49914863,  0.01411715]), -1.0, False, {})
(array([-0.41860276,  0.01704612]), -1.0, False, {})
(array([-0.33239882,  0.01697563]), -1.0, False, {})
(array([-0.25483218,  0.01422817]), -1.0, False, {})
(array([-0.19701395,  0.00959519]), -1.0, False, {})
(array([-0.1657919 ,  0.00392149]), -1.0, False, {})
(array([-0.16426959, -0.00211762]), -1.0, False, {})
(array([-0.19258694, -0.0079639 ]), -1.0, False, {})
(array([-0.24796712, -0.01299252]), -1.0, False, {})
(array([-0.32397854, -0.01640001]), -1.0, False, {})
(array([-0.40992551, -0.01733887]), -1.0, False, {})
(array([-0.49172331, -0.01530338]), -1.0, False, {})
(array([-0.55474537, -0.01047678]), -1.0, False, {})
(array([-0.58740244, -0.00372196]), -1.0, False, {})
(array([-0.58371238,  0.00371146]), -1.0, False, {})
(array([-0.54434798,  0.0104682 ]), -1.0, False, {})
(array([-0.47653219,  0.01529831]), -1.0, Fals

In [6]:
# Initialize environment and reset it to start at valley with 0 velocity
env = gym.make('MountainCar-v0')
env.reset() # array([-0.44943939,  0.        ])

# experiment with constantly moving in one direction or the other to see what happens
# constantly go left
const_action = Push_Left
for t in range(100):
    update = env.step(const_action)
    if t % 5 == 0:
        print(update)
    
# if you constantly go left,  your position stays in (-0.85,-0.48)
# your velocity stays in (-0.015,+0.011)

(array([-0.48093781, -0.00132897]), -1.0, False, {})
(array([-0.50683037, -0.00763126]), -1.0, False, {})
(array([-0.56044358, -0.01252674]), -1.0, False, {})
(array([-0.63191597, -0.01512756]), -1.0, False, {})
(array([-0.70844489, -0.01507362]), -1.0, False, {})
(array([-0.77729269, -0.01261827]), -1.0, False, {})
(array([-0.82834378, -0.00842423]), -1.0, False, {})
(array([-0.8551691 , -0.00323992]), -1.0, False, {})
(array([-0.85481448,  0.00228469]), -1.0, False, {})
(array([-0.82731692,  0.00757176]), -1.0, False, {})
(array([-0.77571274,  0.01199062]), -1.0, False, {})
(array([-0.70652649,  0.01480313]), -1.0, False, {})
(array([-0.6299614 ,  0.01530956]), -1.0, False, {})
(array([-0.55879376,  0.01316345]), -1.0, False, {})
(array([-0.50578696,  0.00861602]), -1.0, False, {})
(array([-0.48069322,  0.0024797 ]), -1.0, False, {})
(array([-0.48813244, -0.00411222]), -1.0, False, {})
(array([-0.52673704, -0.0099479 ]), -1.0, False, {})
(array([-0.58939177, -0.01394959]), -1.0, Fals

In [7]:
print('Action space: ', env.action_space)
n_actions = env.action_space.n

Action space:  Discrete(3)


In [8]:
'''
To descretize the state space, we separate the range of possible continuous positions x and
continuous velocities v into 20 bins (one_feature)
'''

n_feature_bins = 20 # number of state per one feature

env_low = env.observation_space.low     
env_high = env.observation_space.high   
env_distance = (env_high - env_low) / n_feature_bins 

print(env_low, env_high, env_distance, env_distance[0]*20, env_distance[1]*20)
# the range of x is 1.8 and velocity is 0.14 

n_states = n_feature_bins*n_feature_bins # states are the space of combinations of x and v

[-1.2  -0.07] [0.6  0.07] [0.09  0.007] 1.8000000715255737 0.14000000432133675


in the update, the tuple  (array([-0.66511328, -0.01531442]), -1.0, False, {})
 contains the (state, reward, end_of_episode_bool, meta_data)
the meta_data is empty for this environment, there is a negative reward for every timestep that 
 you have not reached the goal yet. 

In [9]:
# load the expert 20 demonstrations
#expert_demo = np.load('/shared/Carson/RL/lets-do-irl/mountaincar/maxent/expert_demo/expert_demo.npy')
expert_demo = np.load('Data/expert_demo.npy')
print(expert_demo.shape)
# (number of demonstrations, length of demonstrations, states and actions of demonstrations)
print(expert_demo[0,60,:], expert_demo[0,0,:].shape)
# as you can see from step 60 of the first example, the best strategy is to first accelerate backwards into
# the < -0.8 range into order to gain speed going right into the valley. 

(20, 130, 3)
[-0.90691623 -0.02983074  0.        ] (3,)


In [10]:
'''
using this bin size we descretize the expert demonstrations
The difference between expert_demo and demonstrations is that
demonstrations consists of discreet integers instead of continuous values
for it's states



here I used n for sample index and t for timestep index
'''
def idx_state(state):
    '''
    this function converts a continuous state vector of
    2-dim into a discrete index
    
    by assigning each state and index of state_idx = position_idx + velocity_idx * one_feature
    we make sure that (position_idx, velocity_idx) = (2,3) and (3,2) map to different integers
    The first 20 elements of state_idx go to position_idx = 0 - 19, velocity_idx = 0, etc
    '''
    position_idx = int((state[0] - env_low[0]) / env_distance[0])
    velocity_idx = int((state[1] - env_low[1]) / env_distance[1])
    state_idx = position_idx + velocity_idx * n_feature_bins
    return state_idx

demonstrations = np.zeros((len(expert_demo), len(expert_demo[0]), 3))

for n in range(len(expert_demo)):
    
    for t in range(len(expert_demo[0])):

        state_idx = idx_state(expert_demo[n][t])

        demonstrations[n][t][0] = state_idx
        demonstrations[n][t][1] = expert_demo[n][t][2] 

Q-Learning is a “model-free, off-policy” RL algorithm.

“model-free” because we are not trying to build a pre-determined model of our environment (which may or may not be accurate)

“off-policy” because our behavior in interacting with (or exploring) the environment may be unrelated to what we believe to be the best optimal at the time of taking the action.

You might think of Q-learning as updating a table where each row is a different state and each column is an action that can be taken in that state and where they intersect is the Q-value for that state action pair. the Q-table maps various (state, action) pairs with the Q-value (the expected sum of discounter future rewards when taking action a in state s and behaving optimally thereafter)

It doesnt have to be a square table like this though, Q just has to map some state or state-action to some Q-value

In the above example you can see that the pattern is that we observe some state, s, take an action a, and receive a new state s’, then repeat. 

The update rule is the weighted average of the old Q(s,a) value and the Q value implied by the new observation. That is, the sum of (i) the immediate reward and (ii) the expected discounted reward received from the new state onwards, assuming you always choose the optimal action.

Q’(s,a) = (1 — alpha) * Q(s, a) + alpha *(r + gamma * Q(s’, argmax a’ : Q(s’, a’)))

where argmax a’ : Q(s’, a’) is the action  a’ that has the highest current Q value among our row of states s’

This equation for update is equivalently written 

𝑄(𝑠,𝑎)←𝑄(𝑠,𝑎)+𝛼(𝑟+𝛾*max_𝑎′𝑄(𝑠′,𝑎′)−𝑄(𝑠,𝑎)) 


In [11]:
q_table = np.zeros((n_states, n_actions)) # (400, 3)

def update_q_table(state, action, reward, q_learning_rate, gamma, next_state):
    ''' 
    The Q-learning update rule
    𝑄(𝑠,𝑎)←𝑄(𝑠,𝑎)+𝛼(𝑟+𝛾*max_𝑎′𝑄(𝑠′,𝑎′)−𝑄(𝑠,𝑎))
    𝛾 or gamma, is the discount factor
    𝛼 or alpha, is the q_learning_rate
    '''
    q_1 = q_table[state][action]
    #print(reward, gamma, max(q_table[next_state])) 
    q_2 = reward + gamma * max(q_table[next_state])
    q_table[state][action] += q_learning_rate * (q_2 - q_1)

# What is Inverse Reinforcement Learning (IRL)

In IRL, instead of randomly exploring the world learning by trial and error, we want to approximate the reward function that experts are using by observing expert demonstrations. This reward function is parameterized by theta

if $f_si$ is the feature vector of some state_i, then the reward = theta.dot(f_si)

In [12]:
def get_reward(feature_matrix, theta, n_states, state_idx):
    
    '''
    theta is the same length as the number of states, so this
    just a list of the rewards associated with being in each 
    state and the function here simply selects the
    element in theta that corresponds to the sate
    '''
    
    irl_rewards = feature_matrix.dot(theta).reshape((n_states,))
    
    return irl_rewards[state_idx]

# What is the Objective Function in MaxEntIRL?

Maximizing the entropy of the distribution over paths subject to the feature constraints from observed data implies that we maximize the likelihood of the observed data under the maximum entropy (exponential family) distribution.

The Product of Probabilites vs The Log Sum of Probabilities

$
\log \left(\prod_i P(x_i)\right) = \sum_i \log \left( P(x_i)\right)
$

the logarithm is a monotonic transformation that preserves the locations of the extrema, in particular, the estimated parameters in maximum-likelihood are identical for the original and the log-transformed formulation, while also morenumerically stable and symbolically easier to differentiate than the former since the sum rule is easier than the product rule for differentiation

Find the Reward Parameters that maximize the likelihood of the expert trajectories, theta_star

$ 
\theta^{*} = \underset{\theta}{\arg\max} \underset{demonstrations}\prod[P(trajectories|\theta)] = \underset{\theta}{\arg\max} \underset{demonstrations}\sum log[P(trajectories|\theta)] = \underset{\theta}{\arg\max} L(\theta)
$

The gradient for this Loss function L is

$\nabla L(\theta) = f_{expert} - \underset{trajectories}\sum P(trajectory|\theta)f_{learner}
= f_{expert} - \underset{states}\sum D_{s} f_{s}
$

Where D_s is the expected state visitation frequencies

gradient = expected_expert_feats - expected_learner_feats

In [13]:
def update_theta(expert, learner, theta, theta_learning_rate):
    '''
    The gradient is the difference between expected
    empirical feature counts and the learner’s expected feature
    counts, which can be expressed in terms of expected state
    visitation frequencies, D_si
    
    the two lines below apply these two formulas
    
    ∇𝐿(𝜃)=𝑓_𝑒𝑥𝑝𝑒𝑟𝑡−∑𝑡𝑟𝑎𝑗𝑒𝑐𝑡𝑜𝑟𝑖𝑒𝑠𝑃(𝑡𝑟𝑎𝑗𝑒𝑐𝑡𝑜𝑟𝑦|𝜃)𝑓_𝑙𝑒𝑎𝑟𝑛𝑒𝑟=𝑓_𝑒𝑥𝑝𝑒𝑟𝑡−∑𝑠𝑡𝑎𝑡𝑒𝑠𝐷𝑠𝑓𝑠
    
    𝜃 := 𝜃 + 𝛼∇𝐿(𝜃)
    
    where 𝛼 is the theta_learning_rate, not the q-learning learning rate
    '''
    gradient = expert - learner
    theta += theta_learning_rate * gradient

    # Clip theta
    for j in range(len(theta)):
        if theta[j] > 0:
            theta[j] = 0

In [14]:
feature_matrix = np.eye((n_states)) # (400, 400)

print(feature_matrix)

def get_expert_feature_expectations(feature_matrix, demonstrations):
    '''
    f_tilda = (1/m)*sum all trajectory's feature count
    Since our states are encoded as indices for 400 possible states, 
    this function adds together one-hot encodings of
    every state visited by each expert demonstration throughout the
    example trajectory and at the end divides the total number of 
    example trajectories (expert demonstrations)
    
    for each 400 states indices, aka each (x,v) pairs, 
    value of 0.5 at that index means that 50% of all trajectories have this state
    '''
    feature_expectations = np.zeros(feature_matrix.shape[0])
    
    for demonstration in demonstrations:
        for state_idx, _, _ in demonstration:
            feature_expectations += feature_matrix[int(state_idx)]

    feature_expectations /= demonstrations.shape[0]
    return feature_expectations

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [ ]:
np.random.seed(1)

'''
Theta θ is the vector containing the reward weights. 
reward(state_t) = np.dot(θ,f(state_t)), where f is the feature vector of state_t
'''
theta_learning_rate = 0.05
theta = -(np.random.uniform(size=(n_states,)))
print(theta[:8], theta.shape)
gamma = 0.99
q_learning_rate = 0.03

expected_expert_feats = get_expert_feature_expectations(feature_matrix, demonstrations) # (400,)
print(expected_expert_feats[370:380], expected_expert_feats.shape)

'''
initialize the learner_feature_counts that when divided by the number of updates 
becomes he
'''
learner_feature_counts = np.zeros(n_states)

episodes, scores = [], []

for episode in range(30000):
    state = env.reset()
    score = 0

    if (episode != 0 and episode == 10000) or (episode > 10000 and episode % 5000 == 0):
        
        expected_expert_feats = learner_feature_counts / episode
        
        update_theta(expected_expert_feats, expected_expert_feats, theta, theta_learning_rate)

    while True:
        state_idx = idx_state(state)
        action = np.argmax(q_table[state_idx])
        next_state, reward, done, _ = env.step(action)

        irl_reward = get_reward(feature_matrix, theta, n_states, state_idx)
        next_state_idx = idx_state(next_state)
        update_q_table(state_idx, action, irl_reward, q_learning_rate, gamma, next_state_idx)

        learner_feature_counts += feature_matrix[int(state_idx)]

        score += reward
        state = next_state

        if done:
            scores.append(score)
            episodes.append(episode)
            break

    if episode % 1000 == 0:
        score_avg = np.mean(scores)
        print('{} episode score is {:.2f}'.format(episode, score_avg))
        pylab.plot(episodes, scores, 'b')
        pylab.savefig("Data/maxent_30000.png")
        np.save("Data/maxent_q_table", arr=q_table)

[-4.17022005e-01 -7.20324493e-01 -1.14374817e-04 -3.02332573e-01
 -1.46755891e-01 -9.23385948e-02 -1.86260211e-01 -3.45560727e-01] (400,)
[0.8  0.65 0.45 0.1  0.   0.   0.   0.   0.   3.4 ] (400,)
0 episode score is -200.00
1000 episode score is -183.53
2000 episode score is -182.77
3000 episode score is -177.25
4000 episode score is -177.33
5000 episode score is -176.36
6000 episode score is -175.94
7000 episode score is -174.84
8000 episode score is -174.34
9000 episode score is -173.21
10000 episode score is -171.69
11000 episode score is -170.09
12000 episode score is -168.73
13000 episode score is -167.53
14000 episode score is -166.52
15000 episode score is -165.66
16000 episode score is -164.90
17000 episode score is -164.22
18000 episode score is -163.63
